## Installing the OpenAI Whisper ASR model

In [1]:
try:
  # Getting whisper
  !pip install -U openai-whisper
  # Getting the kathbath training and evaluation datasets
  # print('Getting the training dataset .. ')
  # !wget https://objectstore.e2enetworks.net/indic-asr-public/indicwhisper/vistaar/kathbath.zip
  # print('Getting the evaluation dataset .. ')
  # !wget https://objectstore.e2enetworks.net/indic-asr-public/indicwhisper/vistaar_benchmarks/kathbath.zip
  # Get the HuggingFace datasets library
#   !pip install datasets
except Exception as e:
  print(f'Couldn\'t install because of error : {e}')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 4.4 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.1/168.1 MB 9.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 59.1 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=801358 sha256=fdb4807233a6d25ce7a5f106ba918c8a61be122b21d4465c5bb142f4a0a62d21
  Stored in directory: /root/.cache/pip/wheels/d0/85/e1/9361b4cbea7dd4b7f6702fa4c3afc94877952eeb2b62f45f56
Successfully built openai-whisper


## Importing all the necessary libraries and frameworks

In [2]:
try:
  import torch
  from torch import nn
  from datasets import load_dataset
except Exception as e:
  print(f"Couldn't import due to : {e}")

## Checking if CUDA is available

In [3]:
# Setting to GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'You are on : {device}')

You are on : cuda


## Importing the Whisper ASR model

In [4]:
import whisper
try:
  variant = input('Enter variant name : ')
  variant = variant.lower()

  model = whisper.load_model(variant)
except Exception as e:
  print(f"Cannot import the {variant} variant for whisper due to : {e}")

Enter variant name :  large


100%|██████████████████████████████████████| 2.88G/2.88G [00:17<00:00, 175MiB/s]


## Getting the Kathbath testing dataset for evaluation

In [5]:
# Getting the kathbath hindi test dataset
try:
  print('Getting the test unknown dataset......')
  !wget https://indic-asr-public.objectstore.e2enetworks.net/indic-superb/kathbath/clean/testunk_audio.tar
  print('Getting the test know dataset......')
  !wget https://indic-asr-public.objectstore.e2enetworks.net/indic-superb/kathbath/clean/testkn_audio.tar
  print('Downloaded both the datasets successfully')
  print('Getting the transcripts....')
  !wget https://indic-asr-public.objectstore.e2enetworks.net/indic-superb/kathbath/clean/transcripts_n2w.tar
except Exception as e:
  print(f'Couldn\'t download due to : {e}')

Getting the test unknown dataset......
--2024-04-26 06:56:26--  https://indic-asr-public.objectstore.e2enetworks.net/indic-superb/kathbath/clean/testunk_audio.tar
Resolving indic-asr-public.objectstore.e2enetworks.net (indic-asr-public.objectstore.e2enetworks.net)... 101.53.152.33, 101.53.136.19, 164.52.210.57, ...
Connecting to indic-asr-public.objectstore.e2enetworks.net (indic-asr-public.objectstore.e2enetworks.net)|101.53.152.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2207580160 (2.1G) [application/x-tar]
Saving to: 'testunk_audio.tar'

testunk_audio.tar   100%[===================>]   2.06G  9.55MB/s    in 2m 56s  

2024-04-26 06:59:24 (12.0 MB/s) - 'testunk_audio.tar' saved [2207580160/2207580160]

Getting the test know dataset......
--2024-04-26 06:59:25--  https://indic-asr-public.objectstore.e2enetworks.net/indic-superb/kathbath/clean/testkn_audio.tar
Resolving indic-asr-public.objectstore.e2enetworks.net (indic-asr-public.objectstore.e2enetwor

## Unzipping and deleting the zip files afterwards for better space management

In [8]:
# Unzipping the audio and the transcript files
# !tar -xzvf /kaggle/working/testkn_audio.tar
# !tar -xzvf /kaggle/working/testunk_audio.tar
# !tar -xzvf /kaggle/working/transcripts_n2w.tar

!tar -xf /kaggle/working/testkn_audio.tar
!tar -xf /kaggle/working/testunk_audio.tar
!tar -xf /kaggle/working/transcripts_n2w.tar

In [9]:
# Deleting the tar files to free up some space
!rm -rf testunk_audio.tar
!rm -rf testkn_audio.tar
!rm -rf transcripts_n2w.tar

## Importing and checking out the ground truths of all the valid audio files

In [11]:
# Now as the transcripts and the audio are not in the same order we will evalute only on the first 10 audios and their corresponding transcripts
import pandas as pd
import os

# Use the code below only in google colab
# ground_truths_df = pd.read_csv('/content/kb_data_clean_m4a/hindi/test_known/transcription_n2w.txt', sep="\t", names=["audio_file", "transcript"])

ground_truths_df = pd.read_csv('/kaggle/working/kb_data_clean_m4a/hindi/test_known/transcription_n2w.txt', sep="\t", names=["audio_file", "transcript"])

In [12]:
ground_truths_df.head()

,audio_file,transcript
0,844424932719996-727-f.m4a,देश चाहता है कि शहीदों को सच्ची श्रद्धांजलि दी...
1,844424930867504-326-f.m4a,इस अप्रत्याशित घटना और नौकरी जाने से दुखी अरवि...
2,844424932746943-645-f.m4a,राजस्थान को पहला झटका अजिंक्य रहाणे के रूप में...
3,844424932732959-645-f.m4a,संग्रहालय देखने के बाद अंसारी अंगकोर वाट मंदिर गए
4,844424933058718-888-f.m4a,पर ऐसा कुछ नहीं है कि सिर्फ ऐसी फिल्में करूंगी


#### Checking the number of files so that we don't miss any during transcription

In [13]:
# Let us explore a bit more about the ground truths file
ground_truths_df.describe()

,audio_file,transcript
count,3130,3130
unique,3130,3130
top,844424933485953-38-m.m4a,द अनुंजियो की प्रसिद्धि बहुत है किंतु उसकी रचन...
freq,1,1


## Steps for the experiment as conducted below :

#### 1. First check if transcription works correctly on first 200 files and GPU usage doesn't kill the kernel

In [15]:
# There seem to be a lot of audio files (3130) due to GPU constraints we will only evaluate a small subset of them
# Let's make a subset conataining about 200 files for our evaluation purposes on the whisper-large asr model

ground_truths_subset = ground_truths_df[:200]
ground_truths_subset.describe()

,audio_file,transcript
count,200,200
unique,200,200
top,844424932781457-645-f.m4a,शब्दों से ज्यादा व्यक्ति के हावभाव उसकी पहचान ...
freq,1,1


In [20]:
# Now let us start transcribing the 200 files in the subset of the df

# First saving all the names of the files to transcribe 
path = list(ground_truths_subset['audio_file'])
print(path)

# The path list contains all the names of the files we will transcribe, moving forward

['844424932719996-727-f.m4a', '844424930867504-326-f.m4a', '844424932746943-645-f.m4a', '844424932732959-645-f.m4a', '844424933058718-888-f.m4a', '844424930829742-326-f.m4a', '844424931080569-229-f.m4a', '844424932609515-645-f.m4a', '844424932400812-384-f.m4a', '844424930680237-3-f.m4a', '844424931189588-229-f.m4a', '844424930975776-326-f.m4a', '844424932335079-384-f.m4a', '844424931013032-1102-f.m4a', '844424932426983-384-f.m4a', '844424931066496-326-f.m4a', '844424932348730-384-f.m4a', '844424932903449-1102-f.m4a', '844424930690568-277-f.m4a', '844424930891371-326-f.m4a', '844424932851606-3-f.m4a', '844424930858559-229-f.m4a', '844424932811689-645-f.m4a', '844424932429563-384-f.m4a', '844424932457819-1102-f.m4a', '844424930948806-277-f.m4a', '844424931078305-229-f.m4a', '844424932594356-645-f.m4a', '844424931010567-229-f.m4a', '844424930639253-326-f.m4a', '844424932814297-3-f.m4a', '844424932350995-384-f.m4a', '844424932906774-727-f.m4a', '844424931128253-229-f.m4a', '844424932396613

In [23]:
# We will get the transcripts of the files like below and then save them along with the predicted transcripts for calculating the WER score for the overall subset as well as the individual scores for each files against their predictions by the Whisper-large
# This will get us the transcript
audio_file='844424932719996-727-f.m4a'
a = ground_truths_subset.loc[ground_truths_subset["audio_file"]==audio_file, 'transcript'].values[0]
print(f'Ground truth for the audio file {audio_file} is {a}')

Ground truth for the audio file 844424932719996-727-f.m4a is देश चाहता है कि शहीदों को सच्ची श्रद्धांजलि दी जाए


In [24]:
# Creating a list for saving all the transcription results
predictions = []

In [26]:
# Transcribing the files and saving the results
for audio_file in path:
  if audio_file.endswith(".m4a"):
    audio_path = '/kaggle/working/kb_data_clean_m4a/hindi/test_known/audio' + '/' + audio_file
    predict = model.transcribe(audio_path, language='hi')
    ground_truth_transcript = ground_truths_subset.loc[ground_truths_subset["audio_file"] == audio_file, 'transcript'].values[0]
    predictions.append({"audio_file" : audio_file, "predicted" : predict['text'], "ground_truth" : ground_truth_transcript})

#### 2. After successful transcription save the predicted values to a dataframe along with the ground_truths

In [27]:
# Save all the predictions in a dataframe
predictions_df = pd.DataFrame(predictions)

In [29]:
# This is how the DF now looks like after predictions
predictions_df.head()

,audio_file,predicted,ground_truth
0,844424932719996-727-f.m4a,देश चाहता है कि शहीदों को सच्चे शद्धांचली दी ...,देश चाहता है कि शहीदों को सच्ची श्रद्धांजलि दी...
1,844424930867504-326-f.m4a,इस प्रत्याशत घटना और नौकरी जाने से दुखी अर्वि...,इस अप्रत्याशित घटना और नौकरी जाने से दुखी अरवि...
2,844424932746943-645-f.m4a,राजस्थान को पहला जटका अजिंग के रहाने के रूप म...,राजस्थान को पहला झटका अजिंक्य रहाणे के रूप में...
3,844424932732959-645-f.m4a,संगराले देखने के बाद अन्सारी अंकोर वाट मंदिर गए।,संग्रहालय देखने के बाद अंसारी अंगकोर वाट मंदिर गए
4,844424933058718-888-f.m4a,पर ऐसा कुछ नहीं है कि सिर्फ ऐसी फिल्में करूँगी,पर ऐसा कुछ नहीं है कि सिर्फ ऐसी फिल्में करूंगी


#### 3. Save current progress to a CSV

In [30]:
# Let's save it to a csv file now
predictions_df.to_csv('predictions_subset.csv', index=False)

#### 4. Check for WER for the first 200 transcriptions

In [31]:
# Now to calculate the wer we will install jiwer
try:
    !pip install jiwer
    from jiwer import wer
except Exception as e:
    print(f"Couldn't import because of : {e}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 10.9 MB/s eta 0:00:0000:010:01


#### 5. Save the WER along with audio file name, predicted transcription, ground truth for easy access and individual score for all the sentences

In [32]:
# After successfully installing and importing let us calculate the WER for the subset
# Get predicted transcripts 
predicted = predictions_df['predicted'].tolist()
# Get ground truth transcripts
ground_truth = predictions_df['ground_truth'].to_list()

# Calculating overall WER for the ground_truths vs predicted transcripts
ovrwer = wer(ground_truth, predicted)
print(f"The Overall WER is : {ovrwer}")

# Calculating the WER for each audio file
# Initializing an empty list to get ground truths
wer_per_file = []
for g, p in zip(ground_truth, predicted):
    file_wer = wer(g, p)
    wer_per_file.append(file_wer)
predictions_df["wer"] = wer_per_file

# Saving as a CSV finally
predictions_df.to_csv('predictions_subset_wer.csv', index=False)

The Overall WER is : 0.24748322147651006


In [33]:
# Let's check out the predictions file
df = pd.read_csv('/kaggle/working/predictions_subset_wer.csv')
df.head()

,audio_file,predicted,ground_truth,wer
0,844424932719996-727-f.m4a,देश चाहता है कि शहीदों को सच्चे शद्धांचली दी ...,देश चाहता है कि शहीदों को सच्ची श्रद्धांजलि दी...,0.300000
1,844424930867504-326-f.m4a,इस प्रत्याशत घटना और नौकरी जाने से दुखी अर्वि...,इस अप्रत्याशित घटना और नौकरी जाने से दुखी अरवि...,0.307692
2,844424932746943-645-f.m4a,राजस्थान को पहला जटका अजिंग के रहाने के रूप म...,राजस्थान को पहला झटका अजिंक्य रहाणे के रूप में...,0.400000
3,844424932732959-645-f.m4a,संगराले देखने के बाद अन्सारी अंकोर वाट मंदिर गए।,संग्रहालय देखने के बाद अंसारी अंगकोर वाट मंदिर गए,0.444444
4,844424933058718-888-f.m4a,पर ऐसा कुछ नहीं है कि सिर्फ ऐसी फिल्में करूँगी,पर ऐसा कुछ नहीं है कि सिर्फ ऐसी फिल्में करूंगी,0.100000


In [34]:
# Let's check the number of values
df.describe()

,wer
count,200.000000
mean,0.249497
std,0.160670
min,0.000000
25%,0.121528
50%,0.222222
75%,0.339286
max,0.700000


#### 6. Repeat the experiment for all the remaining audio files

In [35]:
# Now let us try and run the experiment successfully for the whole dataset at once
ground_truths_remaining = ground_truths_df[200:]
ground_truths_remaining.head()

,audio_file,transcript
200,844424932906752-727-f.m4a,करनाल अर्बन ओडीएफ प्लस भी हो गया है
201,844424932483546-1102-f.m4a,कपड़ाजर्मन अश्लीलधोखाधड़ीपर फिल्माया गया एक वे...
202,844424932907210-1102-f.m4a,सीएम ने कहा कि यह अनुमंडल सूबे के अन्य जिलों क...
203,844424932919894-888-f.m4a,वकील सिंह बाजीगर वर्तमान में दूधवाली ढाणी में ...
204,844424931061593-229-f.m4a,दमकलकर्मियों को सबसे पहले गांव शाहपुर में आग ल...


In [36]:
# Checking the number of values in the df
ground_truths_remaining.describe()

,audio_file,transcript
count,2930,2930
unique,2930,2930
top,844424933485953-38-m.m4a,द अनुंजियो की प्रसिद्धि बहुत है किंतु उसकी रचन...
freq,1,1


# Now let us run the transcription on all of the remaining audio files in the Kathbath dataset

In [37]:
# First saving all the names of the files to transcribe in the path list again
path = list(ground_truths_remaining['audio_file'])
print(path)

['844424932906752-727-f.m4a', '844424932483546-1102-f.m4a', '844424932907210-1102-f.m4a', '844424932919894-888-f.m4a', '844424931061593-229-f.m4a', '844424931069923-326-f.m4a', '844424930972020-326-f.m4a', '844424932760220-3-f.m4a', '844424932759171-468-f.m4a', '844424932814251-3-f.m4a', '844424932349765-384-f.m4a', '844424932802354-468-f.m4a', '844424931004105-326-f.m4a', '844424930909237-326-f.m4a', '844424930865371-326-f.m4a', '844424932736815-645-f.m4a', '844424932609581-645-f.m4a', '844424930931306-384-f.m4a', '844424932781486-645-f.m4a', '844424931048223-384-f.m4a', '844424932795816-645-f.m4a', '844424930583405-326-f.m4a', '844424931275040-1102-f.m4a', '844424932804723-727-f.m4a', '844424931282016-277-f.m4a', '844424932430396-384-f.m4a', '844424930913944-326-f.m4a', '844424932701631-645-f.m4a', '844424932735030-645-f.m4a', '844424930667565-229-f.m4a', '844424930791955-229-f.m4a', '844424932760288-3-f.m4a', '844424931065673-229-f.m4a', '844424930913920-326-f.m4a', '844424932907603

In [38]:
# Re-initializing predictions to an empty list
predictions = []

In [39]:
# Transcribing the remaining files and saving the results
try:
    for audio_file in path:
      if audio_file.endswith(".m4a"):
        audio_path = '/kaggle/working/kb_data_clean_m4a/hindi/test_known/audio' + '/' + audio_file
        predict = model.transcribe(audio_path, language='hi')
        ground_truth_transcript = ground_truths_remaining.loc[ground_truths_remaining["audio_file"] == audio_file, 'transcript'].values[0]
        predictions.append({"audio_file" : audio_file, "predicted" : predict['text'], "ground_truth" : ground_truth_transcript})
except Exception as e:
    print(f"Couldn't complete the transcription due to {e}")

In [40]:
# Now let us create a dataframe for storing the predictions
predictionsfinal_df = pd.DataFrame(predictions)

In [42]:
predictionsfinal_df

,audio_file,predicted,ground_truth
0,844424932906752-727-f.m4a,करनाल अर्बन ODF प्लस भी हो गया है,करनाल अर्बन ओडीएफ प्लस भी हो गया है
1,844424932483546-1102-f.m4a,कपड़ा जर्मन अश्लील दोकाधरी पर फिल्माया गया एक...,कपड़ाजर्मन अश्लीलधोखाधड़ीपर फिल्माया गया एक वे...
2,844424932907210-1102-f.m4a,CM ने कहा कि यह अनुमंडल सूबे के अन्य जिलों के...,सीएम ने कहा कि यह अनुमंडल सूबे के अन्य जिलों क...
3,844424932919894-888-f.m4a,वकील सिंग्बाजीकर वर्तमान में दूद वाली धाड़ी म...,वकील सिंह बाजीगर वर्तमान में दूधवाली ढाणी में ...
4,844424931061593-229-f.m4a,दमकल कर्मियों को सबसे पहले गाउ शाहपूर में आग ...,दमकलकर्मियों को सबसे पहले गांव शाहपुर में आग ल...
...,...,...,...
2925,844424933489380-38-m.m4a,कोरव पांड़व दुन्नों वंस काल की कैती से परस पर...,कौरव पाण्डव दोनों वंश काल की गति से परस्पर लड़...
2926,844424933474857-38-m.m4a,सुक्रोवार को शुरू होने वाला एक समझ्य वर्ष था,शुक्रवार को शुरू होने वाला एक सामान्य वर्ष था
2927,844424933487460-38-m.m4a,स्कूल में उनको दो बार डबल प्रमोशन मिल रहा था,स्कूल में उनको दो बार डबल प्रमोशन मिला था
2928,844424933459397-38-m.m4a,पहला है गर्ल्स ओन टॉप जहां उसने अपनी सबसे बड़...,पहला है गर्ल्स ऑन टॉप जहां उसने अपनी सबसे बड़ी...


In [43]:
# Saving the df to a csv
predictionsfinal_df.to_csv('predicted_remaining.csv', index=False)

In [45]:
# Calculating the WER
predicted = predictionsfinal_df['predicted'].tolist()
ground_truth = predictionsfinal_df['ground_truth'].tolist()

calc_wer = wer(ground_truth, predicted)
print(f"Overall WER is {calc_wer}")

# Calculating per audio file 
wer_per_file = []
for g, p in zip(ground_truth, predicted):
  file_wer = wer(g, p)
  wer_per_file.append(file_wer)
predictionsfinal_df["wer"] = wer_per_file

# Save as CSV
predictionsfinal_df.to_csv('predictionsfinal_wer.csv', index=False)

Overall WER is 0.3006461421512733


#### 7. Combine into a dataframe containing the results for all the audio files in the dataset

In [47]:
# Checking the data
df_final = pd.read_csv('predictionsfinal_wer.csv')
df_final

,audio_file,predicted,ground_truth,wer
0,844424932906752-727-f.m4a,करनाल अर्बन ODF प्लस भी हो गया है,करनाल अर्बन ओडीएफ प्लस भी हो गया है,0.125000
1,844424932483546-1102-f.m4a,कपड़ा जर्मन अश्लील दोकाधरी पर फिल्माया गया एक...,कपड़ाजर्मन अश्लीलधोखाधड़ीपर फिल्माया गया एक वे...,1.642857
2,844424932907210-1102-f.m4a,CM ने कहा कि यह अनुमंडल सूबे के अन्य जिलों के...,सीएम ने कहा कि यह अनुमंडल सूबे के अन्य जिलों क...,0.133333
3,844424932919894-888-f.m4a,वकील सिंग्बाजीकर वर्तमान में दूद वाली धाड़ी म...,वकील सिंह बाजीगर वर्तमान में दूधवाली ढाणी में ...,0.500000
4,844424931061593-229-f.m4a,दमकल कर्मियों को सबसे पहले गाउ शाहपूर में आग ...,दमकलकर्मियों को सबसे पहले गांव शाहपुर में आग ल...,0.500000
...,...,...,...,...
2925,844424933489380-38-m.m4a,कोरव पांड़व दुन्नों वंस काल की कैती से परस पर...,कौरव पाण्डव दोनों वंश काल की गति से परस्पर लड़...,0.714286
2926,844424933474857-38-m.m4a,सुक्रोवार को शुरू होने वाला एक समझ्य वर्ष था,शुक्रवार को शुरू होने वाला एक सामान्य वर्ष था,0.222222
2927,844424933487460-38-m.m4a,स्कूल में उनको दो बार डबल प्रमोशन मिल रहा था,स्कूल में उनको दो बार डबल प्रमोशन मिला था,0.222222
2928,844424933459397-38-m.m4a,पहला है गर्ल्स ओन टॉप जहां उसने अपनी सबसे बड़...,पहला है गर्ल्स ऑन टॉप जहां उसने अपनी सबसे बड़ी...,0.142857


In [49]:
# Combining the first 200 and the final 2930 predictions to a single dataframe
df1 = predictions_df

df2 = predictionsfinal_df

final_df = pd.concat([df1, df2], ignore_index=True)

final_df.describe()

,wer
count,3130.000000
mean,0.297985
std,0.188544
min,0.000000
25%,0.166667
50%,0.272727
75%,0.400000
max,1.642857


## We successfully have all the 3130 audio files and their data in a single dataframe now!

#### 8. Save the results in a CSV

In [50]:
# Getting the WER of the whole dataset
# Get predicted transcripts 
predicted = final_df['predicted'].tolist()
# Get ground truth transcripts
ground_truth = final_df['ground_truth'].to_list()

# Calculating overall WER for the ground_truths vs predicted transcripts
ovrwer = wer(ground_truth, predicted)
print(f"The Overall WER is : {ovrwer}")

# Calculating the WER for each audio file
# Initializing an empty list to get ground truths
wer_per_file = []
for g, p in zip(ground_truth, predicted):
    file_wer = wer(g, p)
    wer_per_file.append(file_wer)
final_df["wer"] = wer_per_file

# Saving as a CSV finally
final_df.to_csv('kathbath_hindi_wer.csv', index=False)

The Overall WER is : 0.29718205920135565


In [51]:
WER_FILE_WISE = pd.read_csv('/kaggle/working/kathbath_hindi_wer.csv')
WER_FILE_WISE.head(10)

,audio_file,predicted,ground_truth,wer
0,844424932719996-727-f.m4a,देश चाहता है कि शहीदों को सच्चे शद्धांचली दी ...,देश चाहता है कि शहीदों को सच्ची श्रद्धांजलि दी...,0.300000
1,844424930867504-326-f.m4a,इस प्रत्याशत घटना और नौकरी जाने से दुखी अर्वि...,इस अप्रत्याशित घटना और नौकरी जाने से दुखी अरवि...,0.307692
2,844424932746943-645-f.m4a,राजस्थान को पहला जटका अजिंग के रहाने के रूप म...,राजस्थान को पहला झटका अजिंक्य रहाणे के रूप में...,0.400000
3,844424932732959-645-f.m4a,संगराले देखने के बाद अन्सारी अंकोर वाट मंदिर गए।,संग्रहालय देखने के बाद अंसारी अंगकोर वाट मंदिर गए,0.444444
4,844424933058718-888-f.m4a,पर ऐसा कुछ नहीं है कि सिर्फ ऐसी फिल्में करूँगी,पर ऐसा कुछ नहीं है कि सिर्फ ऐसी फिल्में करूंगी,0.100000
5,844424930829742-326-f.m4a,आडवानी की यत्रियात्रा नागपूर से गुजरेतो संग ह...,आडवाणी की रथयात्रा नागपुर से गुजरे तो संघ हेडक...,0.636364
6,844424931080569-229-f.m4a,नंगल डैम से सहारनपूर के मध्य चलने वाली मैमू ट...,नंगल डैम से सहारनपुर के मध्य चलने वाली मेमू ट्...,0.214286
7,844424932609515-645-f.m4a,धोनी चोटिल होने की वज़ा से एशिया कप नहीं खेल ...,धोनी चोटिल होने की वजह से एशिया कप में नहीं खे...,0.153846
8,844424932400812-384-f.m4a,जर्मन आक्रमन कारियों ने थियेटर के इंटीरियर और...,जर्मन आक्रमणकारियों ने थिएटर के इंटीरियर और सज...,0.285714
9,844424930680237-3-f.m4a,इसके लिए इसका कमजोर प्रदर्शन भी कम जिम्मेदार ...,इसके लिए इसका कमजोर प्रदर्शन भी कम जिम्मेदार न...,0.000000
